In [1]:
from transformers import pipeline

In [2]:
import pandas as pd

In [3]:
import os

In [4]:
df = pd.read_csv("health_data.csv")

In [5]:
df.head()

,text,disease,severity
0,Patient has mild asthma with occasional cough,asthma,mild
1,Severe pneumonia detected after X-ray,pneumonia,severe
2,Flu symptoms are moderate and improving,flu,moderate
3,Patient reports mild headache and fatigue,headache,mild
4,No sign of fever or cold,none,none


In [6]:
import torch

In [7]:
model_name = "google/flan-t5-base"

In [8]:
# make pipeline
pipe = pipeline(
    "text2text-generation",
    model=model_name,
    device_map="auto",
    dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    max_new_tokens=128
)

Device set to use cpu


Zero-Shot Prompting

In [9]:
def zero_shot_prompt(text):
    prompt = f"""
You are a medical information extractor.
Read this sentence and extract:
- Disease name
- Severity (mild / moderate / severe / none)

Sentence: {text}
Output in this exact format:
Disease: <name>
Severity: <mild/moderate/severe/none>
"""
    result = pipe(prompt, do_sample=False)[0]["generated_text"]
    return result

# test on first 5
for i, row in df.head(5).iterrows():
    print("🩺 Text:", row["text"])
    print("🤖 Model Output:", zero_shot_prompt(row["text"]))
    print("-"*60)

🩺 Text: Patient has mild asthma with occasional cough
🤖 Model Output: Name: name> Severity: mild/moderate/severe/none>
------------------------------------------------------------
🩺 Text: Severe pneumonia detected after X-ray
🤖 Model Output: - Disease name - Severity (mild / moderate / severe / none)
------------------------------------------------------------
🩺 Text: Flu symptoms are moderate and improving
🤖 Model Output: Name: name> Severity: mild/moderate/severe/none>
------------------------------------------------------------
🩺 Text: Patient reports mild headache and fatigue
🤖 Model Output: Name: name> Severity: mild/moderate/severe/none>
------------------------------------------------------------
🩺 Text: No sign of fever or cold
🤖 Model Output: Name: name> Severity: mild/moderate/severe/none>
------------------------------------------------------------


one-shot prompting

In [10]:
def one_shot_prompt(text):
    example_input = "Patient has mild asthma with occasional cough."
    example_output = "Disease: asthma\nSeverity: mild"

    prompt = f"""
You are a medical information extraction assistant.

Example:
Sentence: "{example_input}"
Answer:
{example_output}

Now analyze this new sentence:
Sentence: "{text}"
Answer:
"""
    result = pipe(prompt, do_sample=False)[0]["generated_text"]
    return result

for i, row in df.head(5).iterrows():
    print("🩺 Text:", row["text"])
    print("🤖 Model Output:", one_shot_prompt(row["text"]))
    print("-"*60)


🩺 Text: Patient has mild asthma with occasional cough
🤖 Model Output: Disease: asthma
------------------------------------------------------------
🩺 Text: Severe pneumonia detected after X-ray
🤖 Model Output: X-ray
------------------------------------------------------------
🩺 Text: Flu symptoms are moderate and improving
🤖 Model Output: Flu symptoms are moderate and improving
------------------------------------------------------------
🩺 Text: Patient reports mild headache and fatigue
🤖 Model Output: Headache and fatigue
------------------------------------------------------------
🩺 Text: No sign of fever or cold
🤖 Model Output: No sign of fever or cold
------------------------------------------------------------


few-shot prompting

In [11]:
def few_shot_prompt(text):
    examples = [
        ('Patient has mild asthma with occasional cough.', 'Disease: asthma\nSeverity: mild'),
        ('Severe pneumonia detected after X-ray.', 'Disease: pneumonia\nSeverity: severe'),
        ('Flu symptoms are moderate and improving.', 'Disease: flu\nSeverity: moderate'),
        ('Patient reports mild headache and fatigue.', 'Disease: headache\nSeverity: mild'),
        ('No sign of fever or cold.', 'Disease: none\nSeverity: none')
    ]
    example_block = "\n\n".join([f"Sentence: {i}\nAnswer:\n{o}" for i,o in examples])

    prompt = f"""
You are a medical information extraction assistant.
Learn from these examples and answer in the same style.

{example_block}

New sentence: "{text}"
Answer:
"""
    return pipe(prompt, do_sample=False)[0]["generated_text"]

for i, row in df.head(5).iterrows():
    print("🩺 Text:", row["text"])
    print("🤖 Model Output:", few_shot_prompt(row["text"]))
    print("-"*60)


🩺 Text: Patient has mild asthma with occasional cough
🤖 Model Output: Disease: asthma
------------------------------------------------------------
🩺 Text: Severe pneumonia detected after X-ray
🤖 Model Output: Disease: pneumonia
------------------------------------------------------------
🩺 Text: Flu symptoms are moderate and improving
🤖 Model Output: Disease: flu
------------------------------------------------------------
🩺 Text: Patient reports mild headache and fatigue
🤖 Model Output: Disease: headache
------------------------------------------------------------
🩺 Text: No sign of fever or cold
🤖 Model Output: Disease: none
------------------------------------------------------------


In [12]:
def few_shot_prompt_v2(text):
    examples = [
        ('Patient has mild asthma with occasional cough.', 'Disease: asthma\nSeverity: mild'),
        ('Severe pneumonia detected after X-ray.', 'Disease: pneumonia\nSeverity: severe'),
        ('Flu symptoms are moderate and improving.', 'Disease: flu\nSeverity: moderate'),
        ('Patient reports mild headache and fatigue.', 'Disease: headache\nSeverity: mild'),
        ('No sign of fever or cold.', 'Disease: none\nSeverity: none')
    ]

    # build example block
    example_block = "\n\n".join(
        [f"Sentence: {inp}\nAnswer:\n{out}" for inp, out in examples]
    )

    prompt = f"""
You are a clinical information extraction assistant.
Your task: From each medical sentence, identify both the disease name
and its severity level (mild / moderate / severe / none).

Follow the examples below carefully:

{example_block}

Now analyze this new sentence:
Sentence: "{text}"

Answer format (exactly two lines):
Disease: <disease or none>
Severity: <mild / moderate / severe / none>
"""

    result = pipe(prompt, do_sample=False, max_new_tokens=128)[0]["generated_text"]
    return result


# --- run on the same 5 test cases ---
for i, row in df.head(5).iterrows():
    print("🩺 Text:", row["text"])
    print("🤖 Model Output:", few_shot_prompt_v2(row["text"]))
    print("-" * 60)


🩺 Text: Patient has mild asthma with occasional cough
🤖 Model Output: none
------------------------------------------------------------
🩺 Text: Severe pneumonia detected after X-ray
🤖 Model Output: none
------------------------------------------------------------
🩺 Text: Flu symptoms are moderate and improving
🤖 Model Output: none
------------------------------------------------------------
🩺 Text: Patient reports mild headache and fatigue
🤖 Model Output: none
------------------------------------------------------------
🩺 Text: No sign of fever or cold
🤖 Model Output: none
------------------------------------------------------------


Load large model 

In [13]:
model_name = "google/flan-t5-large"

In [14]:
pipe = pipeline(
    "text2text-generation",
    model=model_name,
    device_map="auto",
    dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    max_new_tokens=128
)

C:\Users\Administrator.DESKTOP-U6N2VRV\venv\Lib\site-packages\accelerate\utils\modeling.py:1614: UserWarning: The following device_map keys do not match any submodules in the model: ['decoder.embed_tokens', 'encoder.embed_tokens']
  warnings.warn(
Some parameters are on the meta device because they were offloaded to the cpu and disk.
Device set to use cpu


In [15]:
def few_shot_prompt_v2(text):
    examples = [
        ('Patient has mild asthma with occasional cough.', 'Disease: asthma\nSeverity: mild'),
        ('Severe pneumonia detected after X-ray.', 'Disease: pneumonia\nSeverity: severe'),
        ('Flu symptoms are moderate and improving.', 'Disease: flu\nSeverity: moderate'),
        ('Patient reports mild headache and fatigue.', 'Disease: headache\nSeverity: mild'),
        ('No sign of fever or cold.', 'Disease: none\nSeverity: none')
    ]

    # build example block
    example_block = "\n\n".join(
        [f"Sentence: {inp}\nAnswer:\n{out}" for inp, out in examples]
    )

    prompt = f"""
You are a clinical information extraction assistant.
Your task: From each medical sentence, identify both the disease name
and its severity level (mild / moderate / severe / none).

Follow the examples below carefully:

{example_block}

Now analyze this new sentence:
Sentence: "{text}"

Answer format (exactly two lines):
Disease: <disease or none>
Severity: <mild / moderate / severe / none>
"""

    result = pipe(prompt, do_sample=False, max_new_tokens=128)[0]["generated_text"]
    return result


# --- run on the same 5 test cases ---
for i, row in df.head(5).iterrows():
    print("🩺 Text:", row["text"])
    print("🤖 Model Output:", few_shot_prompt_v2(row["text"]))
    print("-" * 60)


🩺 Text: Patient has mild asthma with occasional cough
🤖 Model Output: asthma
------------------------------------------------------------
🩺 Text: Severe pneumonia detected after X-ray
🤖 Model Output: pneumonia
------------------------------------------------------------
🩺 Text: Flu symptoms are moderate and improving
🤖 Model Output: flu
------------------------------------------------------------
🩺 Text: Patient reports mild headache and fatigue
🤖 Model Output: headache Severity: mild
------------------------------------------------------------
🩺 Text: No sign of fever or cold
🤖 Model Output: none
------------------------------------------------------------


In [19]:
def extract_disease(text):
    prompt = f"""
You are a medical information extractor.
Identify the disease or condition mentioned in this sentence.
If none is mentioned, answer 'none'.

Sentence: {text}
Answer:
"""
    result = pipe(prompt, do_sample=True, temperature=0.7)[0]["generated_text"]
    return result.strip()

def extract_severity(text):
    prompt = f"""
You are a medical information extractor.
Determine the severity level (mild / moderate / severe / none) mentioned in this sentence.
If not stated, answer 'none'.

Sentence: {text}
Answer:
"""
    result = pipe(prompt, do_sample=True, temperature=0.7)[0]["generated_text"]
    return result.strip()

# ----------------------------------------------------
# 4️⃣ Run the pipeline on a few examples
# ----------------------------------------------------
for i, row in df.head(5).iterrows():
    text = row["text"]
    disease = extract_disease(text)
    severity = extract_severity(text)

    print(f"🩺 Text: {text}")
    print(f"🤖 Disease: {disease}")
    print(f"🤖 Severity: {severity}")
    print("-" * 60)

🩺 Text: Patient has mild asthma with occasional cough
🤖 Disease: asthma
🤖 Severity: mild
------------------------------------------------------------
🩺 Text: Severe pneumonia detected after X-ray
🤖 Disease: pneumonia
🤖 Severity: severe
------------------------------------------------------------
🩺 Text: Flu symptoms are moderate and improving
🤖 Disease: Flu
🤖 Severity: moderate
------------------------------------------------------------
🩺 Text: Patient reports mild headache and fatigue
🤖 Disease: None
🤖 Severity: mild
------------------------------------------------------------
🩺 Text: No sign of fever or cold
🤖 Disease: None
🤖 Severity: none
------------------------------------------------------------


In [21]:
from sklearn.metrics import accuracy_score

pred_disease = []
pred_severity = []

for text in df["text"]:
    disease = extract_disease(text)
    severity = extract_severity(text)
    pred_disease.append(disease.lower())
    pred_severity.append(severity.lower())

# clean true labels
true_disease = df["disease"].str.lower().tolist()
true_severity = df["severity"].str.lower().tolist()

disease_acc = accuracy_score(true_disease, pred_disease)
severity_acc = accuracy_score(true_severity, pred_severity)

print(f"✅ Disease accuracy:  {disease_acc*100:.1f}%")
print(f"✅ Severity accuracy: {severity_acc*100:.1f}%")


✅ Disease accuracy:  70.0%
✅ Severity accuracy: 100.0%


In [22]:
results_df = df.copy()
results_df["pred_disease"] = pred_disease
results_df["pred_severity"] = pred_severity

results_df.to_csv("results_flan_t5_large.csv", index=False)
print("✅ Results saved to results_flan_t5_large.csv")
results_df.head()


✅ Results saved to results_flan_t5_large.csv


,text,disease,severity,pred_disease,pred_severity
0,Patient has mild asthma with occasional cough,asthma,mild,asthma,mild
1,Severe pneumonia detected after X-ray,pneumonia,severe,pneumonia,severe
2,Flu symptoms are moderate and improving,flu,moderate,flu,moderate
3,Patient reports mild headache and fatigue,headache,mild,none,mild
4,No sign of fever or cold,none,none,none,none


In [23]:
report = f"""
Clinical Information Extraction — Prompt-Based Experiment
----------------------------------------------------------
Model: {model_name}
Samples: {len(df)}
Disease accuracy:  {disease_acc*100:.1f} %
Severity accuracy: {severity_acc*100:.1f} %

Notes:
- Zero-shot → model repeated template
- One-shot → partial extraction
- Few-shot → consistent disease extraction
- Two-step prompting + sampling → improved severity

Conclusion:
Using open-source FLAN-T5 models, prompt engineering alone
achieves meaningful structured extraction for short clinical
notes, mirroring the early stages of Shan Wang et al. (2024).

Generated automatically by your notebook.
"""

with open("project_report.txt", "w", encoding="utf-8") as f:
    f.write(report)

print(report)



Clinical Information Extraction — Prompt-Based Experiment
----------------------------------------------------------
Model: google/flan-t5-large
Samples: 20
Disease accuracy:  70.0 %
Severity accuracy: 100.0 %

Notes:
- Zero-shot → model repeated template
- One-shot → partial extraction
- Few-shot → consistent disease extraction
- Two-step prompting + sampling → improved severity

Conclusion:
Using open-source FLAN-T5 models, prompt engineering alone
achieves meaningful structured extraction for short clinical
notes, mirroring the early stages of Shan Wang et al. (2024).

Generated automatically by your notebook.

